# TEST run on local via clean scripts

In [ ]:
import sys, os, errno

import numpy as np
import time, datetime
import copy, joblib, shutil
from collections import Counter

module_path = os.path.dirname(os.getcwd())+'/../src'
if module_path not in sys.path:
    sys.path.append(module_path)

#SEED_tf=42  #=0
#import tensorflow as tf
#tf.compat.v1.set_random_seed(SEED_tf) 

import keras.backend as K

from light_curve import LightCurve
import functions_preprocess as m_preprocess
import functions_keras as m_func
#import nets_ae_clf as m_nets #m_dln

SEED=0
np.random.seed(SEED)

In [ ]:
import functions_run_last as m_runs

In [5]:
%run ../src/setup_notebook

E0615 19:19:37.322119 140734833571264 execution.py:657] File `'../src/setup_notebook.py'` not found.


In [ ]:
m_preprocess.print_versions()

In [ ]:
import importlib

## - Args





        dict_nruns = {0 : 'classifier_MLP_meta',
                      #
                      1 : 'classifier_direct_RNN',
                      2 : 'classifier_direct_tCNN',
                      3 : 'classifier_direct_dTCN',
                      #
                      4 : 'autoencoder_RNN',
                      5 : 'autoencoder_tCNN',
                      6 : 'autoencoder_dTCN',
                      #
                      7 : 'composite_net_RNN',
                      8 : 'composite_net_tCNN',
                      9 : 'composite_net_dTCN',
                     }

In [ ]:
#data_store         = '/Users/sjamal/Downloads/local_disk_store/'
data_store         = '/Users/sjamal/git/deepnet_vs/data/MACHO/'
output_store       = '/Users/sjamal/git/deepnet_vs/outputs/trained_models/'

data_id            = 'blue'
 
nb_epoch           = 200 #200

## ------------------------------------------------------------------------------------ ##
## ------------------------------------------------------------------------------------ ##
model_type         = 'tCNN'
run_id             = 8
## ------------------------------------------------------------------------------------ ##
## ------------------------------------------------------------------------------------ ##


sim_type           = m_runs.dict_nruns[run_id]

sizenet            = 16
num_layers         = 1
drop_frac          = 0.25
batch_size         = 128
learning_rate      = 5e-4

validation_split   = 0.20

gpu_frac           = 0.00

m_causal           = True
m_categorical      = True

m_embedding        = 8 if run_id in np.r_[m_runs.list_ae, m_runs.list_composite] else None


m_raw              = False         ## normalized data versus initial values
m_padding          = True#         ## fixed-length data versus initial data
m_fold             = True          ## phase-folded verus time series

m_meta             = True

m_reductionfactor  = 2

## dTCN
if run_id in [3,6,9]:
    n_stacks       = num_layers
    kernel_wavenet = 1
    max_dilation   = 2
    kernel_size    = 3
    m_activation   = 'wavenet'
## tCNN
if run_id in [2,5,8]:
    max_dilation   = 2
    kernel_size    = 5
    m_activation   = 'tanh'
## RNN (LSTM; GRU)
if run_id in [1,4,7]:
    bidirectional  = True


arg_dict = {'data_id'           : data_id,
            'run_id'            : run_id ,
            #
            'data_store'        : data_store ,
            'output_store'      : output_store,
            #
            'sizenet'           : sizenet ,
            'num_layers'        : num_layers, 
            'drop_frac'         : drop_frac ,
            'batch_size'        : batch_size,
            'nb_epoch'          : nb_epoch,
            'model_type'        : model_type, 
            'learning_rate'     : learning_rate ,
            #
            'embedding'         : m_embedding,
            'add_dense'         : True ,
            'validation_split'  : validation_split ,
            #
            'categorical'       : m_categorical,
            'causal'            : m_causal ,
            'sim_type'          : sim_type+('_fixedlength' if m_padding else '_generator'),
            'gpu_frac'          : gpu_frac, 
            #
            'add_metadata'      : m_meta,
            #
            'loss_AE'           : 'mae',
            'loss_CLF'          : 'categorical_crossentropy' if m_categorical else 'logcosh',
            'metrics_CLF'       : 'categorical_accuracy'     if m_categorical else 'accuracy',
            #
            'use_raw'           : m_raw,        # normalized data versus initial values
            'padding'           : m_padding ,   # fixed-length data versus initial data
            'period_fold'       : m_fold ,      # phase-folded verus time series
            #
            'no_train'          : False
           }

## RNN (LSTM; GRU)
if run_id in [1,4,7]:
    arg_dict['bidirectional']     = bidirectional      # rnn
## tCNN
if run_id in [2,5,8]:
    arg_dict['m_activation']      = m_activation
    arg_dict['kernel_size']       = kernel_size
    arg_dict['max_dilation']      = max_dilation
## dTCN
if run_id in [3,6,9]:    
    arg_dict['m_reductionfactor'] = m_reductionfactor  # tcnn
    arg_dict['m_activation']      = m_activation
    arg_dict['kernel_size']       = kernel_size
    arg_dict['kernel_wavenet']    = kernel_wavenet
    arg_dict['n_stacks']          = n_stacks
    arg_dict['max_dilation']      = max_dilation
#


args = m_func.parse_model_args(arg_dict)

## ############## 1 - Set args session ##############  ##

In [ ]:
#importlib.reload(m_runs); importlib.reload(m_func)

In [ ]:
arg_dict = m_runs.set_params_cline(args)

## ############## 2 - Load stored data_structures  ############## ##

In [ ]:
#importlib.reload(m_runs)

In [ ]:
input_lcs=None; input_metadata=None

input_lcs, input_metadata, output_dict = m_runs.get_data(arg_dict)

In [ ]:
print('Metadata :', input_metadata['selected'].columns)

## ############## 3 - Train network(s) ############## ##

In [ ]:
#importlib.reload(m_nets); 
importlib.reload(m_func); importlib.reload(m_runs);

In [ ]:
stime = time.time() 

if True:
    m_runs.run_autoencoder(arg_dict, input_lcs, input_metadata, output_dict)
    
hours, rem = divmod(time.time() - stime, 3600) #timeit.default_timer()-stime
minutes, seconds = divmod(rem, 60)
print("\n*Execution time : {:0>2} h {:0>2} min {:05.2f} s".format(int(hours), int(minutes), seconds))

In [3]:
import sys, os, errno

import numpy as np
import time, datetime
import copy, joblib, shutil
from collections import Counter

module_path = os.path.dirname(os.getcwd())+'/../src'
if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
import functions_results

Using TensorFlow backend.
